In [3]:
from simulation_files.libs.libraries import *

In [ ]:
geolocator = Nominatim(user_agent="em")
geocode = partial(geolocator.geocode, language="en")
#location = geolocator.geocode("paris")
loc = geocode("paris")

In [4]:
loc = Location("Paris, Île-de-France, France", (48.8566969, 2.3514616), raw= {})

In [ ]:
print(loc.address)
print((loc.latitude, loc.longitude))

Paris, Île-de-France, France
(48.8566969, 2.3514616)


In [ ]:
bss = pd.read_csv("datasets/Paris_4G_Sites_Lat_Long.csv")

In [ ]:
bss.head()

,id_station_anfr,latitude,longitude
0,752290181,48.89722,2.31305
1,752290285,48.83750,2.34805
2,752290316,48.89667,2.33056
3,752290341,48.89750,2.33861
4,752290284,48.83778,2.35861


In [ ]:
m = folium.Map([loc.latitude, loc.longitude], zoom_start=12)

for i, row in bss.iterrows():
    #print(row)
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row["id_station_anfr"],
        icon=folium.DivIcon(html=f"""<div style="font-size: 8px; color: blue;"><i class="fa fa-caret-up"></i></div>""")
        #icon=folium.DivIcon(html=f"""<div style="font-size: 5px; color: red;"><i class="fa fa-circle"></i></div>""")
        #icon=folium.DivIcon(html=f"""<div style="color: black; font-size: 16px; font-family: 'Arial';"><i class="fa fa-caret-up"></i></div>""")
    ).add_to(m) # add the marker to the map


legend_html = '''
<div style="position: fixed; 
            top: 25px; right: 25px; width: 100px; height: 60px; 
            border:2px solid grey; z-index:9999; font-size:10px;
            background-color: white; opacity: 0.8; padding: 5px;
            ">&nbsp; Legend <br>
            &nbsp; <span style="font-size: 10px;"><i class="fa fa-caret-up" style="color: blue;"></i>&nbsp; Base Station <br>
            &nbsp; <span style="height: 6px; width: 6px; background-color: red; border-radius: 50%; display: inline-block;"></span>&nbsp; User <br>
</div>
'''
#&nbsp; <span style="font-size: 10px;"><i class="fa fa-circle" style="color: red;"></i>&nbsp; User <br>
# Add the legend HTML to the map
m.get_root().html.add_child(folium.Element(legend_html));

In [ ]:
m

In [ ]:
m.save("output/map.html")

In [ ]:
#bufferino
import folium

# Create the base map
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Adding base stations from your DataFrame 'bss'
for i, row in bss.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row["id_station_anfr"],
        icon=folium.DivIcon(html=f"""<div style="font-size: 8px; color: blue;"><i class="fa fa-caret-up"></i></div>""")
    ).add_to(m)

# Example data for user trajectories
trajectories = {
    'user1': [(48.8566, 2.3522), (48.8570, 2.3527), (48.8575, 2.3532)],
    'user2': [(48.8560, 2.3512), (48.8565, 2.3517), (48.8570, 2.3522)],
    # Add as many users as needed
}

# Plotting each user's trajectory and adding a marker for the last known location
for user, path in trajectories.items():
    # Draw the trajectory
    folium.PolyLine(path, color='red', weight=2.5, opacity=1).add_to(m)
    # Add a marker for the last position in the path
    folium.Marker(
        location=path[-1],  # Using the last tuple in the list as the current location
        popup=f"{user}",
        icon=folium.DivIcon(html=f"""<div style="font-size: 8px; color: red;"><i class="fa fa-circle"></i></div>""")
    ).add_to(m)

# Legend HTML as defined before
legend_html = '''
<div style="position: fixed; 
            top: 25px; right: 25px; width: 100px; height: 60px; 
            border:2px solid grey; z-index:9999; font-size:10px;
            background-color: white; opacity: 0.8; padding: 5px;
            ">&nbsp; Legend <br>
            &nbsp; <span style="font-size: 10px;"><i class="fa fa-caret-up" style="color: blue;"></i>&nbsp; Base Station <br>
            &nbsp; <span style="height: 6px; width: 6px; background-color: red; border-radius: 50%; display: inline-block;"></span>&nbsp; User <br>
</div>
'''

# Add the legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m

*let's build the mobility model*

In [ ]:
df = pd.read_csv("utd19_u.csv")

/var/folders/0h/f5d7gkhd32v8pr7h9szg79wc0000gn/T/ipykernel_8859/1898410669.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("utd19_u.csv")


In [ ]:
df.head()

,day,interval,detid,flow,occ,error,city,speed
0,2017-05-06,0,06.X-2li,12.0,0.0,1.0,augsburg,NaN
1,2017-05-06,300,06.X-2li,12.0,0.0,1.0,augsburg,NaN
2,2017-05-06,600,06.X-2li,12.0,0.0,1.0,augsburg,NaN
3,2017-05-06,900,06.X-2li,16.0,0.0,1.0,augsburg,NaN
4,2017-05-06,1200,06.X-2li,16.0,0.0,1.0,augsburg,NaN


In [ ]:
df[df['city'] == 'paris'].head()

,day,interval,detid,flow,occ,error,city,speed
115990908,2016-01-01,3600,1,2340.0,0.053,NaN,paris,NaN
115990909,2016-01-01,7200,1,2460.0,0.053,NaN,paris,NaN
115990910,2016-01-01,10800,1,2226.0,0.050,NaN,paris,NaN
115990911,2016-01-01,14400,1,1761.0,0.038,NaN,paris,NaN
115990912,2016-01-01,18000,1,1465.5,0.031,NaN,paris,NaN


In [ ]:
df_paris_flow = df[df['city'] == 'paris']

In [ ]:
print('Paris data size:', len(df_paris_flow), 'entries')

Paris data size: 1548314 entries


In [ ]:
print('Date interval:', df_paris_flow['day'].min(), 'to', df_paris_flow['day'].max())

Date interval: 2016-01-01 to 2016-12-01


In [ ]:
print('Number of detid:', len(df_paris_flow['detid'].unique()))

Number of detid: 209


In [ ]:
#@TODO:
""" 
1. Build the mobility model using the random stocastic baseline
2. Build the birth-death process as an upper layer over the mobiliy model
3. Compare the baseline with your mobility model
"""

*Baseline: Reference Point Group Mobility Model (RPGM), adopting the Gauss–Markov Mobility Model (GMM) per entity granualrity*
- ensure cluster movement, i.e. spatial correlation within UEs
- ensure smooth trnasition, i.e. temporal correlation between UEs

In [ ]:
#simulate timeline
T = 100 #s
ues_pool = generate_user(n_users, prob_map, grid) #==>DONE but i need to calculate the probability_map

generate_ues_clusters(n_clusters) #==> DONE

generate_cluster_params(n_clusters) #==> Done

#handle death and birth process thread
time.time()
start process in the background
as t goes each user has an associated clock
so the death is handled in either in the UE class or Controller
the birth is by mathematical euqation

for t in range(T):
    for c in clusters:
        for ue in c.paired_ues:
            ue.move() #==> done
        c.move_destination() #==> done

    if t%T_SAMPLE == 0:
        compute_analytics()
        update_log()

print_log()
plot_simulation2D()

if graphics:
    plot_simulation3D()


In [ ]:
generate a python file for each class to clean the code

In [ ]:
def generate_users(n_users, prob_map, grid):
    """
    Generate users in the network
    """
    users = []
    for _ in range(n_users):
        x, y = np.random.randint(0, NET_WIDTH), np.random.randint(0, NET_HEIGHT)
        while prob_map[x][y] == 0:
            x, y = np.random.randint(0, NET_WIDTH), np.random.randint(0, NET_HEIGHT)
        users.append(UE(x, y, grid[x][y]))
    return users

## Gif rendering

In [ ]:
def initialize_map(center, zoom_start=12):
    return folium.Map(location=center, zoom_start=zoom_start)

def add_base_stations_to_map(map_obj, base_stations):
    for bs in base_stations:
        folium.Marker(
            bs.get_geo_coordinate(),
            popup=f"BS {bs.bs_id}",
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(map_obj)


def update_ue_positions(map_obj, ues):
    # Clear previous UE markers; assumes markers are stored in a feature group or layer control
    for layer in map_obj._children:
        if isinstance(map_obj._children[layer], folium.map.FeatureGroup):
            map_obj._children[layer].clear()
    
    ue_layer = folium.FeatureGroup(name="UEs")
    for ue in ues:
        folium.Marker(
            ue.get_geo_coordinate(),
            popup=f"UE {ue.ue_id} - BS {ue.bs_id}",
            icon=folium.Icon(color='blue', icon='user')
        ).add_to(ue_layer)
    ue_layer.add_to(map_obj)


def save_map_state(map_obj, filename):
    map_obj.save(filename)

def create_gif(image_folder, output_path, duration=0.5):
    images = [imageio.imread(os.path.join(image_folder, filename)) for filename in sorted(os.listdir(image_folder)) if filename.endswith('.png')]
    imageio.mimsave(output_path, images, duration=duration)

def draw_cluster_trajectories(map_obj, clusters):
    for cluster in clusters:
        # Convert each destination in the cluster's history to geographic coordinates
        lat_lon_positions = [ue.get_geo_coordinate(destination) for destination in cluster.history]
        # Draw a line on the map for each cluster's trajectory
        folium.PolyLine(
            lat_lon_positions,
            color=np.random.choice(['red', 'blue', 'green', 'purple', 'orange']),
            weight=2,
            opacity=0.7,
            popup=f"Cluster {cluster.id}"
        ).add_to(map_obj)

def update_map_with_trajectories(map_obj, clusters):
    for cluster in clusters:
        lat_lon_positions = [convert_km_to_geo(cluster.history[i], cluster.history[i+1]) for i in range(len(cluster.history)-1)]
        folium.PolyLine(lat_lon_positions, color='red', weight=3, opacity=0.8).add_to(map_obj)
    map_obj.save('map.html')

def convert_km_to_geo(x, y): #fix this
    # Convert kilometer positions to geographic coordinates assuming a certain origin
    ORIGIN_LAT, ORIGIN_LONG = 48.8566, 2.3522  # Example: Paris coordinates
    return [ORIGIN_LAT + y / 110.574, ORIGIN_LONG + x / (111.320 * math.cos(math.radians(ORIGIN_LAT)))]